# SMR manhattan plot

In [ ]:
suppressMessages({
    library(dplyr)
    library(ggtext)
    library(ggplot2)
})

## Functions

In [ ]:
save_plots <- function(p, fn){
    for(ext in c('.png', '.pdf', '.svg')){
        ggsave(paste0(fn,ext), p, width=30, height=15, units="cm")
    }
}

## Prepare data

In [ ]:
annot_file = "/ceph/projects/v4_phase3_paper/inputs/counts/gene_annotation/_m/gene_annotation.tsv"
gene_annot <- data.table::fread(annot_file) %>% janitor::clean_names()
gene_annot %>% head(2)

### Preparing SMR data

In [ ]:
smr_file = "../../_m/eqtl_genes.eqtl_p1e-04.gwas_p5e-08.csv"
qsmr_data_load <- data.table::fread(smr_file) %>% janitor::clean_names() %>%
    filter(fdr < 0.05, p_heidi > 0.01) %>% select(probe_id, probe_bp, fdr) %>% distinct

smr_data_load <- data.table::fread(smr_file) %>% janitor::clean_names() %>%
    inner_join(gene_annot, by=c("probe_id"="feature_id")) %>%
    select(probe_id, probe_bp, probe_chr, p_smr, p_heidi, top_snp, symbol) %>% distinct %>%
    rename(chr=probe_chr)

data_cum <- smr_data_load %>% group_by(chr) %>% summarise(max_bp = max(probe_bp)) %>%
    mutate(bp_add = lag(cumsum(as.numeric(max_bp)), default=0)) %>% select(chr, bp_add)
smr_data <- smr_data_load %>% inner_join(data_cum, by="chr") %>% 
    mutate(bp_cum = probe_bp + bp_add) %>% left_join(qsmr_data_load, by=c("probe_id", "probe_bp")) %>% 
    mutate(fdr = tidyr::replace_na(fdr, 1))
smr_data %>% head(2)

### SMR caudate, SZ PGC3

### Plotting settings

In [ ]:
axis_set <- smr_data %>% group_by(chr) %>% summarize(center = mean(bp_cum))
ylim <- smr_data %>% filter(p_smr == min(p_smr)) %>% 
    mutate(ylim  = abs(floor(log10(p_smr))) + 2) %>% pull(ylim)
ylim = ylim[1]
sig <- 1e-4

### Plot

In [ ]:
manhplot1 <- ggplot(smr_data, aes(x=bp_cum, y=-log10(p_smr), size=-log10(p_smr))) + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    geom_hline(yintercept = -log10(sig), color = "grey40", linetype = "dashed") + 
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(0, ylim)) +
    scale_color_manual(values = rep(ggpubr::get_palette("jco", 2), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "-log<sub>10</sub>(p)") +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot1

In [ ]:
save_plots(manhplot1, 'caudate_smr_manhattanplot')